## Fake News Classifier Using LSTM

In this project, I would try to implement LSTM (Long short-term memory) to classify fake news based on the `title` of the news. LSTM is an artificial neural network used in the field of deep learning and is heavily used in NLP based tasks.

The dataset has been taken from kaggle: https://www.kaggle.com/c/fake-news/data#

In [1]:
# Importing Libraries
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Embedding 
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from tensorflow.keras.layers import Dropout

In [2]:
# Checking tensorflow version
tf.__version__

'2.4.1'

In [3]:
# Read the dataset
df=pd.read_csv('train.csv')
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [4]:
# Drop Nan Values
df=df.dropna()
df.reset_index(inplace=True, drop = True)

In [5]:
# Get the Independent Features
X=df.drop('label',axis=1)

In [6]:
# Get the Dependent features
y=df['label']

In [7]:
X.shape

(18285, 4)

In [8]:
y.shape

(18285,)

In [9]:
# Vocabulary size (size of the vocabulary dictionary)
voc_size=5000

### One-hot Representation

The first step here would be to convert the collection of words into a one-hot representation.
But before that, we need to make sure the data is pre-processed.

In [10]:
# creating a copy of X
messages=X.copy()

In [11]:
messages['title'][0]

'House Dem Aide: We Didn’t Even See Comey’s Letter Until Jason Chaffetz Tweeted It'

In [12]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/badvendetta/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [13]:
# Dataset Preprocessing

ps = PorterStemmer()

corpus = []

for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i]) # removing unnecessary characters
    review = review.lower() # converting to lower case
    review = review.split() # removing whitespaces
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')] # stemming the words
    review = ' '.join(review)
    corpus.append(review)

In [14]:
corpus[0]

'hous dem aid even see comey letter jason chaffetz tweet'

In [15]:
# Finally, converting to one-hot representation
onehot_repr=[one_hot(words,voc_size)for words in corpus] 
onehot_repr[0]

[4330, 719, 3705, 3183, 449, 3778, 4380, 944, 1190, 41]

### Embedding Representation

In [16]:
sent_length=20 # max length of sentence we are allowing
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length) # pre-padding used here
print(embedded_docs)

[[   0    0    0 ...  944 1190   41]
 [   0    0    0 ...  394 1225 4390]
 [   0    0    0 ... 2798  530  846]
 ...
 [   0    0    0 ... 1144 2412 2629]
 [   0    0    0 ...  248 3821 2214]
 [   0    0    0 ... 4159 4267  900]]


In [17]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 4330,
        719, 3705, 3183,  449, 3778, 4380,  944, 1190,   41], dtype=int32)

### Building the model

In [18]:
embedding_vector_features=40 # Number of features in the Feature representation
model=Sequential() # Building a sequential model
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length)) # Adding an embedding layer
model.add(LSTM(100)) # Adding an LSTM layer with 100 neurons
model.add(Dense(1,activation='sigmoid')) # Adding a dense layer with sigmoid activation function
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy']) # Compiling the model
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 40)            200000    
_________________________________________________________________
lstm (LSTM)                  (None, 100)               56400     
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [19]:
len(embedded_docs),y.shape

(18285, (18285,))

In [20]:
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [21]:
X_final.shape,y_final.shape

((18285, 20), (18285,))

In [22]:
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.30, random_state=42)

### Model Training

In [23]:
# Model training with 10 epochs (Will take some time!)
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
200/200 [==============================] - 8s 33ms/step - loss: 0.4656 - accuracy: 0.7665 - val_loss: 0.1952 - val_accuracy: 0.9165
Epoch 2/10
200/200 [==============================] - 5s 27ms/step - loss: 0.1299 - accuracy: 0.9483 - val_loss: 0.2004 - val_accuracy: 0.9154
Epoch 3/10
200/200 [==============================] - 6s 30ms/step - loss: 0.0989 - accuracy: 0.9628 - val_loss: 0.2326 - val_accuracy: 0.9160
Epoch 4/10
200/200 [==============================] - 5s 27ms/step - loss: 0.0620 - accuracy: 0.9807 - val_loss: 0.2468 - val_accuracy: 0.9114
Epoch 5/10
200/200 [==============================] - 5s 26ms/step - loss: 0.0479 - accuracy: 0.9822 - val_loss: 0.3246 - val_accuracy: 0.9143
Epoch 6/10
200/200 [==============================] - 5s 26ms/step - loss: 0.0252 - accuracy: 0.9915 - val_loss: 0.3017 - val_accuracy: 0.9038
Epoch 7/10
200/200 [==============================] - 6s 28ms/step - loss: 0.0176 - accuracy: 0.9943 - val_loss: 0.4649 - val_accuracy: 0.9081

### Performance Metrics And Accuracy

In [24]:
y_pred=model.predict_classes(X_test)

accuracy_score(y_test,y_pred)

/Users/badvendetta/anaconda3/lib/python3.8/site-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


0.8966460080204156

In [25]:
confusion_matrix(y_test,y_pred)

array([[2868,  239],
       [ 328, 2051]])

So we got a pretty good accuracy score of 0.89 which is much better than the Logistic regression score of 0.81 which I previously trained. To further improve the score, I tried adding two dropout layers to the sequential model as shown below.

### Adding Dropout 

In [26]:
# Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Dropout(0.3)) # Dropout of 0.3
model.add(LSTM(100))
model.add(Dropout(0.3)) # Dropout of 0.3
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [27]:
# Model training with 10 epochs (Will take some time!)
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
200/200 [==============================] - 11s 40ms/step - loss: 0.4854 - accuracy: 0.7402 - val_loss: 0.1982 - val_accuracy: 0.9131
Epoch 2/10
200/200 [==============================] - 7s 33ms/step - loss: 0.1467 - accuracy: 0.9430 - val_loss: 0.1983 - val_accuracy: 0.9202
Epoch 3/10
200/200 [==============================] - 6s 30ms/step - loss: 0.1040 - accuracy: 0.9622 - val_loss: 0.2105 - val_accuracy: 0.9141
Epoch 4/10
200/200 [==============================] - 6s 30ms/step - loss: 0.0802 - accuracy: 0.9733 - val_loss: 0.2325 - val_accuracy: 0.9180
Epoch 5/10
200/200 [==============================] - 6s 29ms/step - loss: 0.0536 - accuracy: 0.9812 - val_loss: 0.2593 - val_accuracy: 0.9169
Epoch 6/10
200/200 [==============================] - 6s 30ms/step - loss: 0.0425 - accuracy: 0.9861 - val_loss: 0.3790 - val_accuracy: 0.9107
Epoch 7/10
200/200 [==============================] - 6s 30ms/step - loss: 0.0310 - accuracy: 0.9911 - val_loss: 0.4214 - val_accuracy: 0.908

In [28]:
y_pred=model.predict_classes(X_test)

accuracy_score(y_test,y_pred)

0.909952606635071

In [29]:
confusion_matrix(y_test,y_pred)

array([[2808,  299],
       [ 195, 2184]])

## Summary

We were able to classify the fake news titles with a pretty good accuracy of 0.90 on the test data by using LSTM with additional dropout layers. We could also try playing around with some hyperparameters to improve the model accuracy such as vocab size, embedding vector features, etc. 